# Setup

In [2]:
%load_ext autoreload
%autoreload 2

from  import Spatial
import scanpy as sc
import squidpy as sq
import functools
import os
from os import system
import pandas as pd

ddd = os.path.join(os.path.expanduser("~"), "projects/data/public/visium")

pd.options.display.max_columns = 100

# genes = ["CDKN1A", "CDKN2A", "TP53", "PLAUR",
#          "PTGER4", "ATG16L1", "NOD2", "LACC1"]
# cell_types = None
# palette = "tab20"

genes = dict(CSF=["CSF2", "CSF2RB", "CSF2RA"],
             All_ILCs=["IL7R"],
             ILC1=["TBX21", "KLRB1", "CCL5", "HSP90AA1", "IFNG"],
             ILC3=["RORC", "IL23R", "KIT", "CCL20", "TCF7", "IL22"],
             Cytokines=["IFNG", "IL22"])  # ILC1 & ILC3, respectively
cell_types = ["ILC3", "LTi-like NCR+ ILC3", "LTi-like NCR- ILC3",
              "ILCP", "Macrophages", "Stem cells"]
# palette = ["r", "tab:pink", "m", "b", "tab:brown", "cyan"]
palette = "tab10"

kws_pp = dict(cell_filter_pmt=[0, 100], cell_filter_ncounts=None,
              cell_filter_ngene=[0, None], gene_filter_ncell=[3, None],
              gene_filter_ncounts=None)  # can call using 1 arguments

SyntaxError: invalid syntax (3692531250.py, line 4)

# Data

In [ ]:
selves = []
for s in zip(["Post-Xenium"], ["_Post_Xenium"]):
    for i, n in enumerate(["Rep1", "Rep2"]):
        d_p = os.path.join(ddd, f"CytAssist_FFPE_Human_Colon/{s[0]}/{n}")
        self = Spatial(d_p, visium=True, col_cell_type="leiden")
        self.adata.obs = self.adata.obs.assign(Condition=s[0])
        self.adata.obs = self.adata.obs.assign(Replicate=i)
        self.adata.obs = self.adata.obs.assign(Sample=f"{s[0]}_{n}")
        # self._columns["col_sample_id"] = "Sample"
        # self._columns["col_condition"] = "Condition"
        selves += [self]

## Preprocessing

In [ ]:
%%time

_ = self.preprocess(**kws_pp)

## Clustering

In [ ]:
%%time

_ = self.cluster()

# Variables

In [ ]:
cell_types = list(self.rna.obs[self._columns["col_cell_type"]].sample(3))

# Centrality Scores

In [ ]:
%%time

self.calculate_centrality(n_jobs=4)

# Cell Type Co-Occurrence

In [ ]:
%%time

_ = self.find_cooccurrence(key_cell_type=cell_types, figsize=(60, 20),
                           palette=palette, kws_plot=dict(wspace=3))

# Neighbors Enrichment Analysis

In [ ]:
%%time

_ = self.calculate_neighborhood(figsize=(60, 30), cmap="coolwarm",
                                cbar_range=[-30, 30])

# Spatially-Variable Genes

In [ ]:
%%time

ggg = genes if genes and isinstance(genes, dict) else {"Genes": genes}
genes_all = functools.reduce(lambda i, j: i + j, [ggg[g] for g in ggg])
# for g in ggg:
#   self.find_svgs(genes=ggg[g])
fig = self.find_svgs(
    genes=genes_all, method="moran", n_perms=10, kws_plot=dict(
        wspace=0.1, cmap="coolwarm", vcenter=0, size=3,
        legend_fontsize="large"), figsize=(15, 15))

# Receptor-Ligand Interactions

In [ ]:
%%time

out = self.calculate_receptor_ligand(
    key_sources=cell_types, key_targets=cell_types, col_condition=False,
    p_threshold=0.001, remove_ns=True, figsize=(15, 10))

In [ ]:
os.listdir(self._dir)

In [ ]:
self.adata.uns["spatial"]